In [1]:
#import necessary library
import numpy as np # linear algebra
import pandas as pd # data manipulation and analysis
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # data visualization
sns.set_style('whitegrid') # set style for visualization
import warnings # ignore warnings
warnings.filterwarnings('ignore')
from initial_report import *
import ast

In [2]:
#import dataset
df_events=pd.read_csv("events.csv")

In [3]:
#randomly get 5 rows
df_events.sample(5)

,customer_id,event,value,time
158101,d8a7d08af02243ae8ba4206010bb57f9,offer received,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},408
79944,e923430fec954402b739fec4f21d8f8b,offer viewed,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},198
224488,6d66909ef05a413fb279cfd75160d284,transaction,{'amount': 4.36},516
207011,713a1ea744ee4267a741645078845eac,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},504
76938,9bf2feb2b3ba4c439b040a257b71edb1,offer viewed,{'offer id': '3f207df678b143eea3cee63160fa8bed'},186


In [4]:
#get_initial_report
initial_report(df_events)

 *** DATA CLEANING CHECKLIST ***
----------------------------------------
*** Structure:
- Total Rows: 306534
- Total Columns: 4
- Column Names: ['customer_id', 'event', 'value', 'time']

*** Data Types:
  customer_id: object
  event: object
  value: object
  time: int64

*** Mixed Data Types:

*** Distinct Values per Column:
  customer_id: 17000
  event: 4
  value: 5121
  time: 120

*** Null Values and Percentages:


*** Duplicates: 397

*** Negative or Zero Values:
  time: 15561

*** Basic Statistics:
                time
count  306534.000000
mean      366.382940
std       200.326314
min         0.000000
25%       186.000000
50%       408.000000
75%       528.000000
max       714.000000

*** Category Description:
                             customer_id        event  \
count                             306534       306534   
unique                             17000            4   
top     94de646f7b6041228ca7dec82adb97d2  transaction   
freq                                  51       

1. has duplicates
2. has negatives
3. need to check the data type of column-> value
4. check categroy description for event column

In [5]:
#get counts for column: event
df_events.event.value_counts()

event
transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: count, dtype: int64

In [6]:
#see all the duplicates
df_events[df_events.duplicated(keep=False)]

,customer_id,event,value,time
66122,3dde94fa581145cb9f206624f1a94d5a,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,168
66123,3dde94fa581145cb9f206624f1a94d5a,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,168
66782,e9fb6ed2cecb4980ba98c86abc9c91e3,offer completed,{'offer_id': 'ae264e3637204a6fb9bb56bc8210ddfd...,168
66783,e9fb6ed2cecb4980ba98c86abc9c91e3,offer completed,{'offer_id': 'ae264e3637204a6fb9bb56bc8210ddfd...,168
67613,a7dc060f6fc94ca7bf71fbb188187dca,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,168
...,...,...,...,...
304756,0785f1fce0b04ba08e01c7d2ebab4917,offer completed,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7...,708
305550,b7e216b6472b46648272c29a52a86702,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,714
305551,b7e216b6472b46648272c29a52a86702,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,714
306455,f3e801caeafe4899b3b989b586e74ac7,offer completed,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7...,714


In [7]:
#delete duplicates
df_events.drop_duplicates(inplace=True)

In [8]:
df_events.shape

(306137, 4)

In [9]:
#check data type for value
df_events['value'].apply(type).value_counts()

value
<class 'str'>    306137
Name: count, dtype: int64

In [10]:
#convert 'value" to dict
df_events['value'] = df_events['value'].apply(ast.literal_eval)

In [11]:
#again: check data type for value
df_events['value'].apply(type).value_counts()

value
<class 'dict'>    306137
Name: count, dtype: int64

In [12]:
#check list of keys
from itertools import chain

# Create a flat list of all keys used in the 'value' column
all_keys = list(set(chain.from_iterable(df_events['value'].apply(lambda x: x.keys()))))

print(all_keys)


['offer_id', 'offer id', 'reward', 'amount']


1.there are two offer_id; need to standardize

In [13]:
#standardize offer_id keys
def standardize_keys(d):
    return {k.replace(' ', '_').lower(): v for k, v in d.items()}

df_events['value'] = df_events['value'].apply(standardize_keys)


In [14]:
#check again the list
from itertools import chain

# Create a flat list of all keys used in the 'value' column
all_keys = list(set(chain.from_iterable(df_events['value'].apply(lambda x: x.keys()))))

print(all_keys)

['offer_id', 'reward', 'amount']


In [15]:
## Expand the dictionary into separate columns
value_expanded = df_events['value'].apply(pd.Series)

# Join back to the original DataFrame
df_events = df_events.drop('value', axis=1).join(value_expanded)

In [16]:
#check sample
df_events.sample(5)

,customer_id,event,time,offer_id,amount,reward
84742,812ca7f271cb4cf88417ece7a73092f4,transaction,210,NaN,0.79,NaN
260454,d39e704f67fd4b9f98b83b0ed422ef6c,transaction,576,NaN,0.36,NaN
281461,e44d3bde10b147568f9c11fb0414abf5,transaction,618,NaN,25.74,NaN
278311,3c5e22dca5164b0ca1203e2ba2a31ca7,transaction,612,NaN,1.00,NaN
145614,e433442492614997ad025b268178226c,transaction,390,NaN,2.95,NaN


In [17]:
#chheck negative in time column
df_events[df_events.time<0]

,customer_id,event,time,offer_id,amount,reward


1. No negative no problem

In [18]:
#save file
df_events.to_csv('cleaned_events.csv', index=False)

In [19]:
df_events.event.value_counts()

event
transaction        138953
offer received      76277
offer viewed        57725
offer completed     33182
Name: count, dtype: int64